## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [1]:
import numpy as np
import pandas as pd
import pickle

from janitor import clean_names

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
ruta_proyecto = 'C:/Users/alvar/Proyectos ML/03_RIESGOS'

### Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'trabajo.csv'

### Cargar los datos

In [4]:
ruta_completa = ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col=1)

### Seleccionar solo las variables finales

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [5]:
variables_finales = ['ingresos_verificados',
                     'vivienda',
                     'finalidad',
                     'num_cuotas',
                     'antigüedad_empleo',
                     'rating',
                     'ingresos',
                     'dti',
                     'num_lineas_credito',
                     'porc_uso_revolving',
                     'principal',
                     'tipo_interes',
                     'imp_cuota',
                     'num_hipotecas',
                     'porc_tarjetas_75p',
                     'num_cancelaciones_12meses',
                     'num_derogatorios',
                     'estado',
                     'imp_amortizado',
                     'imp_recuperado'
                  ]

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

### Seleccionar variables

Quedarse solo con las de la lista.

In [6]:
df = df[variables_finales]

## ESTRUCTURA DE LOS DATASETS

### limpiar nombres

In [7]:
df = clean_names(df)

### Eliminar registros

#### Por atípicos

In [8]:
num_desv_tip = 4

In [9]:
def atipicos_desv_tip(variable, num_desv_tip = 4):
    #sacamos los nulos por ahora
    variable = variable.dropna()
    #calculamos los límites
    media = np.mean(variable)
    sd = np.std(variable)
    umbral = sd * num_desv_tip
    lim_inf = media - umbral
    lim_sup = media + umbral
    #encontramos los índices de los que están fuera de los límites
    indices = [indice for indice,valor in variable.items() if valor < lim_inf or valor > lim_sup]
    return(indices)

In [10]:
var_atipicos_dt = ['ingresos']

In [11]:
for variable in var_atipicos_dt:
    df.drop(atipicos_desv_tip(df[variable], num_desv_tip),axis = 0, inplace = True)

In [12]:
a_eliminar=df.loc[(df['dti']>100)|(df['dti']<0)].index.values

In [13]:
df=df.loc[~df.index.isin(a_eliminar)]

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [14]:
var_imputar_desconocido = df.select_dtypes(exclude = 'number').columns.to_list()
valor='Desconocido'
var_imputar_moda = df.select_dtypes(include='number').columns.to_list()

In [15]:
def calidad_datos(temp,var_imputar_desconocido,valor,var_imputar_moda):
    for variable in var_imputar_desconocido:
        temp[variable]=temp[variable].fillna(valor)
        
    for variable in var_imputar_moda:
            if pd.api.types.is_integer_dtype(temp[variable]):
                    
                temp[variable] = temp[variable].fillna(int(temp[variable].mode()[0]))
            else:
                temp[variable] = temp[variable].fillna(temp[variable].mode()[0])
    return temp

### Instanciar creación de variables

Dado que la creación de variables es diferente para los 3 modelos necesitamos construir 3 funciones.

#### Crear las funciones

In [16]:
def creacion_variables_pd(temp):
#hacer una copia, sino se sobre escribe df
    temp=df.copy() 
#corregir valores variables
    temp['finalidad']=temp.apply(lambda x:'other' if x.finalidad in['renewable_energy','educational','wedding'] else x.finalidad, axis=1)
    temp['vivienda']=temp['vivienda'].replace(['ANY','OTHER','NONE'],'MORTGAGE')
    temp['porc_uso_revolving']=temp.apply(lambda x: 100 if x.porc_uso_revolving>100 else x.porc_uso_revolving,axis=1)
    temp['num_cancelaciones_12meses']=np.where(temp['num_cancelaciones_12meses']>0,1,0)
    temp['num_derogatorios']=np.where(temp['num_derogatorios']>0,1,0)

#Crear target   
    temp['target_pd']=np.where(temp['estado'].isin(['Charged Off','Does not meet the credit policy. Status:Charged Off','Default']),1,0)

#Eliminamos variables que no usamos en este modelo y la que usamos para crear la target
    temp.drop(columns=['estado','imp_recuperado','imp_amortizado'],inplace=True)

#Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [17]:
def creacion_variables_ead(temp):
#hacer una copia, sino se sobre escribe df
    temp=df.copy() 
#corregir valores variables
    temp['finalidad']=temp.apply(lambda x:'other' if x.finalidad in['renewable_energy','educational','wedding'] else x.finalidad, axis=1)
    temp['vivienda']=temp['vivienda'].replace(['ANY','OTHER','NONE'],'MORTGAGE')
    temp['porc_uso_revolving']=temp.apply(lambda x: 100 if x.porc_uso_revolving>100 else x.porc_uso_revolving,axis=1)
    temp['num_cancelaciones_12meses']=np.where(temp['num_cancelaciones_12meses']>0,1,0)
    temp['num_derogatorios']=np.where(temp['num_derogatorios']>0,1,0)

#Crear target   
    temp['target_ead']=temp.apply(lambda x: ((x['principal']-x['imp_amortizado'])
                                             /x['principal']) if ((x['principal']-x['imp_amortizado'])/x['principal'])>=0 else 0 ,axis=1)

#Eliminamos variables que no usamos en este modelo y la que usamos para crear la target
    temp.drop(columns=['estado','imp_recuperado','imp_amortizado'],inplace=True)

#Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [18]:
def creacion_variables_lgd(temp):
#hacer una copia, sino se sobre escribe df
    temp=df.copy() 
#corregir valores variables
    temp['finalidad']=temp.apply(lambda x:'other' if x.finalidad in['renewable_energy','educational','wedding'] else x.finalidad, axis=1)
    temp['vivienda']=temp['vivienda'].replace(['ANY','OTHER','NONE'],'MORTGAGE')
    temp['porc_uso_revolving']=temp.apply(lambda x: 100 if x.porc_uso_revolving>100 else x.porc_uso_revolving,axis=1)
    temp['num_cancelaciones_12meses']=np.where(temp['num_cancelaciones_12meses']>0,1,0)
    temp['num_derogatorios']=np.where(temp['num_derogatorios']>0,1,0)

#Crear target   
    temp['target_lgd'] = temp.apply(lambda x: 
                              1 - (x['imp_recuperado'] / (x['principal'] - x['imp_amortizado'])) 
                              if (x['principal'] - x['imp_amortizado']) != 0 
                              else np.nan, axis=1)
    temp['target_lgd'] = temp['target_lgd'].apply(lambda x: 0 if np.isnan(x) or np.isinf(x) or x<0 else x)

#Eliminamos variables que no usamos en este modelo y la que usamos para crear la target
    temp.drop(columns=['estado','imp_recuperado','imp_amortizado'],inplace=True)

#Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

#### Crear los dataframes de X e y

In [19]:
x_pd, y_pd = creacion_variables_pd(calidad_datos(df,var_imputar_desconocido,valor,var_imputar_moda))

x_ead, y_ead = creacion_variables_ead(calidad_datos(df,var_imputar_desconocido,valor,var_imputar_moda))

x_lgd, y_lgd = creacion_variables_lgd(calidad_datos(df,var_imputar_desconocido,valor,var_imputar_moda))

In [20]:
x_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139482 entries, 137387967 to 94394801
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ingresos_verificados       139482 non-null  object 
 1   vivienda                   139482 non-null  object 
 2   finalidad                  139482 non-null  object 
 3   num_cuotas                 139482 non-null  object 
 4   antiguedad_empleo          139482 non-null  object 
 5   rating                     139482 non-null  object 
 6   ingresos                   139482 non-null  float64
 7   dti                        139482 non-null  float64
 8   num_lineas_credito         139482 non-null  float64
 9   porc_uso_revolving         139482 non-null  float64
 10  principal                  139482 non-null  float64
 11  tipo_interes               139482 non-null  float64
 12  imp_cuota                  139482 non-null  float64
 13  num_hipotecas      

### Instanciar transformación de variables

In [22]:
#ONE HOT ENCODING
var_ohe = [ 'ingresos_verificados', 'vivienda','finalidad','num_cuotas']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')


#ORDINAL ENCODING
var_oe = ['antiguedad_empleo','rating']

orden_antiguedad_empleo = ['desconocido','< 1 year','1 year','2 years','3 years','4 years',
                           '5 years','6 years','7 years','8 years','9 years','10+ years']

orden_rating = ['G','F','E','D','C','B','A']

oe = OrdinalEncoder(categories = [orden_antiguedad_empleo,orden_rating],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 12)


#MIN-MAX SCALING
var_mms=['antiguedad_empleo','rating']+(x_pd.select_dtypes(include=['number']).columns.to_list())
mms = MinMaxScaler()


In [23]:
var_mms

['antiguedad_empleo',
 'rating',
 'ingresos',
 'dti',
 'num_lineas_credito',
 'porc_uso_revolving',
 'principal',
 'tipo_interes',
 'imp_cuota',
 'num_hipotecas',
 'porc_tarjetas_75p',
 'num_cancelaciones_12meses',
 'num_derogatorios']

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [24]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (oe, var_oe),
    remainder='passthrough')

#### Crear el pipeline de preprocesamiento

In [25]:
pipe_prepo = make_pipeline(ct, mms)

### Instanciar los modelos

#### Instanciar los algoritmos

In [26]:
modelo_pd = LogisticRegression(solver = 'saga', n_jobs=-1, C = 0.25, penalty = 'l1')

modelo_ead = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 10,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0.75)

modelo_lgd = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 20,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0)

#### Crear los pipes finales de entrenamiento

In [27]:
pipe_entrenamiento_pd = make_pipeline(pipe_prepo,modelo_pd)

pipe_entrenamiento_ead = make_pipeline(pipe_prepo,modelo_ead)

pipe_entrenamiento_lgd = make_pipeline(pipe_prepo,modelo_lgd)

#### Guardar el pipe final de entrenamiento

In [28]:
ruta_pipe_entrenamiento_pd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_pd.pickle'

with open(ruta_pipe_entrenamiento_pd, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_pd, file)

In [29]:
ruta_pipe_entrenamiento_ead = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_ead.pickle'

with open(ruta_pipe_entrenamiento_ead, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_ead, file)

In [30]:
ruta_pipe_entrenamiento_lgd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_lgd.pickle'

with open(ruta_pipe_entrenamiento_lgd, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_lgd, file)

#### Entrenar los pipes

In [31]:
pipe_ejecucion_pd = pipe_entrenamiento_pd.fit(x_pd,y_pd)
pipe_ejecucion_ead = pipe_entrenamiento_ead.fit(x_ead,y_ead)
pipe_ejecucion_lgd = pipe_entrenamiento_lgd.fit(x_lgd,y_lgd)

In [32]:
scoring_pd = pipe_ejecucion_pd.predict_proba(x_pd)[:, 1]
ead = pipe_ejecucion_ead.predict(x_ead)
lgd = pipe_ejecucion_lgd.predict(x_lgd)

## GUARDAR EL PIPE

### Guardar el pipe final de ejecución

In [33]:
ruta_pipe_ejecucion_pd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_pd.pickle'

with open(ruta_pipe_ejecucion_pd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_pd, file)

In [34]:
ruta_pipe_ejecucion_ead = ruta_proyecto + '/04_Modelos/pipe_ejecucion_ead.pickle'

with open(ruta_pipe_ejecucion_ead, mode='wb') as file:
   pickle.dump(pipe_ejecucion_ead, file)

In [35]:
ruta_pipe_ejecucion_lgd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_lgd.pickle'

with open(ruta_pipe_ejecucion_lgd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_lgd, file)